In [ ]:
%%capture
!pip install contextualized-topic-models==2.2.0

In [ ]:
%%capture
!pip install pyldavis

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm

In [ ]:
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
stop_words.update(('\r', '\n', '\r\n', '\n\r', "i've", 'ever', 'much', 'say', "i'm", "you're", "they're", "we're", "i'd", "you'd", 'even', 'also', 'look', 'still'))

In [ ]:
imdb = pd.read_csv("/content/drive/MyDrive/imdb_review.csv")
mydrama = pd.read_csv('/content/drive/MyDrive/my_dramalist_crawling.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mydrama = mydrama.drop(['Unnamed: 0.1'], axis=1)
mydrama

,Unnamed: 0,genre,review
0,0,comedy,This drama was such a delight and surprise to ...
1,1,comedy,So this beautiful Romcom has finally come to a...
2,2,comedy,"""Encounter"" is % romance, with no filler, so i..."
3,3,comedy,This drama is a treat for romance drama lovers...
4,4,comedy,This review may contain spoilers\r\nIt will to...
...,...,...,...
8786,8786,action,"Season II, YESSSSS!\r\nI love Lee Seung Gi and..."
8787,8787,action,This review may contain spoilers\r\nOne of the...
8788,8788,action,This review may contain spoilers\r\nTHIS REVIE...
8789,8789,action,Its my first review. There are many good kdram...


In [ ]:
action_review = (imdb['genre'] =='action')
thriller_review = (imdb['genre'] == 'thriller')
comedy_review = (imdb['genre'] == 'comedy')

df_action_review = imdb[action_review]['review']
df_thriller_review = imdb[thriller_review]['review']
df_comedy_review = imdb[comedy_review]['review']

In [ ]:
# df = pd.concat([imdb, mydrama])
# df['genre'].unique()

array(['drama', 'crime', 'comedy', 'thriller', 'romance', 'action',
       'fantasy'], dtype=object)

In [ ]:
# drama_review = (df['genre'] =='drama')
# thriller_review = (df['genre'] == 'thriller')
# comedy_review = (df['genre'] == 'comedy')

# df_drama_review = df[drama_review]['review']
# df_thriller_review = df[thriller_review]['review']
# df_comedy_review = df[comedy_review]['review']

In [ ]:
sentence = df_action_review.to_list()

In [ ]:
sent = ",".join(sentence)

In [ ]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        a = ''.join(a)
        word_tokens = self.tagger(a)
        
        words = []

        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        # pos_words = [word for word in tag_words if word[1][0] in {'V','N','J','R'}]
        pos_words = []
        for word in tag_words:
            if word[1] in ['VB', 'VBP', 'NNP', 'NN', 'JJ']:
                pos_words.append(word)
        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        return results

<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<ipython-input-65-1fc2728e24ad>:12: DeprecationWarning: invalid escape sequence \?
  text = re.sub('[-=+,#/\?:^$.@*

In [ ]:
custom_tokenizer = CustomTokenizer(text_to_word_sequence)

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=4000, ngram_range=(1,2), min_df = 0.05, max_df = 0.1)

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(sentence)

In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [ ]:
# train_contextualized_embeddings = bert_embeddings_from_list(sentence, "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

train_contextualized_embeddings = bert_embeddings_from_list(sentence, 'sentence-transformers/distiluse-base-multilingual-cased-v2')
# train_contextualized_embeddings = bert_embeddings_from_list(sentence, 'sentence-transformers/msmarco-distilbert-dot-v5')

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=512, n_components=5, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [198680/198680]	Train Loss: 4.268972272817244	Time: 0:00:04.370614: : 20it [01:29,  4.49s/it]


In [ ]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['bad', 'first', 'end', 'last', 'see'],
             1: ['zombie', 'make', 'way', 'think', 'lot'],
             2: ['drama', 'action', 'acting', 'love', 'character'],
             3: ['time', 'character', 'many', 'love', 'think'],
             4: ['game', 'squid', 'acting', 'plot', 'think']})

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Sampling: [10/10]: : 10it [00:34,  3.48s/it]
